In [30]:
import string
import pandas as pd
import sparql_dataframe
from rdflib import Graph
import urllib

### organize prefixes

In [31]:
dic_prefixes = {
    "http://purl.org/dc/terms/": 'dcterms' ,
    "https://w3id.org/function/ontology#": 'fno' ,
    "https://w3id.org/function/ontology/n3#": 'fnon' ,
    "http://www.w3.org/2000/10/swap/crypto#": 'crypto' ,
    "http://www.w3.org/2000/10/swap/list#": 'list' ,
    "http://www.w3.org/2000/10/swap/log#": 'log' ,
    "http://www.w3.org/2000/10/swap/math#": 'math' ,
    "http://www.w3.org/1999/02/22-rdf-syntax-ns#": 'rdf' ,
    "http://www.w3.org/2000/01/rdf-schema#": 'rdfs' ,
    "http://www.w3.org/2000/10/swap/string#": 'string' ,
    "http://www.w3.org/2000/10/swap/time#": "time" ,
    "http://www.w3.org/2003/06/sw-vocab-status/ns#": "vc" ,
    "http://www.w3.org/2001/XMLSchema#": "xsd" ,
}

In [32]:
from glob import glob
from os.path import join

g = Graph()
# iterate recursively over all files in the src directory
files = glob(join('src', '**', '*.n3'), recursive=True)
try:
    for file in files:
        g += Graph().parse(file, format='n3')
except:
    print('Error parsing file: ' + file)
    raise

In [33]:
[g.bind(v, k) for k, v in dic_prefixes.items()]

[None, None, None, None, None, None, None, None, None, None, None, None, None]

In [34]:
sparql_prefixes = ' '.join(['prefix ' + v + ': <' + k + '>\n' for k, v in dic_prefixes.items()])

In [35]:
query = """
    select distinct (concat(strbefore(str(?s), '#'), '#') as ?namespace) 
    where { ?s a fno:Function . ?s ?p ?o . }
"""
prefixes = [dic_prefixes[str(row.namespace)] for row in g.query(query)]

### processing

In [36]:
editor_uri = "https://n3-editor.herokuapp.com/n3/editor/?formula="
md_string = ""

In [37]:
for p in prefixes:
    md_string += f"## {p} ##" + " {#" + p + "}\n"
    q = sparql_prefixes + """
        select (strafter(str(?s), '#') as ?name) ?s ?tldr ?comment ?exampleDescription ?example
        where {
            ?s a fno:Function ;
                fno:name ?name ;
                fnon:tldr ?tldr ;
                dcterms:comment ?comment ;
            .
            filter(strstarts(str(?s), "$NAMESPACE"))            
        }
    """
    # name and description
    query = string.Template(q).substitute(NAMESPACE=next(key for key, value in dic_prefixes.items() if value == p))
    for func in g.query(query):
        md_string += "### " + p + ":" + func.name + " ### {#" + str(func.name) + "}\n"
        md_string += func.tldr + "\n\n"
        md_string += func.comment + "\n\n"
        md_string += f"#### Schema(s) #### {{#{func.name}_schemas}} \n"
        # PARAMETERS

        spec_string = ""
        q = sparql_prefixes + """
            select ?positionName ?predicate ?mode ?description 
            where {
                <$FUNCTION> a fno:Function ;
                    fno:parameter ?list .
                    ?list rdf:rest*/rdf:first ?member .
                    ?member a fno:Parameter ;
                        dcterms:description ?description ;
                        fno:mode ?mode ;
                        fno:predicate ?predicate ;
                        fnon:position ?position .
                bind(strafter(str(?position), '#') as ?positionName)
            }
        """
        query = string.Template(q).substitute(FUNCTION=str(func.s))

        for param in g.query(query):
            if str(param.positionName) == "subject":
                subject = f"( {str(param.predicate)} ){str(param.mode)} "
            if str(param.positionName) == "object":
                object = f" {str(param.predicate)}{str(param.mode)} "
        md_string += f"<i>`{subject} {p}:{str(func.name)} {object}`</i>\n\n"

        # examples        
        md_string += f"#### Examples #### {{#{func.name}_examples}}\n"
        query = sparql_prefixes + """
            select ?description ?seeAlso ?expression ?result
            where {
                <$FUNCTION> a fno:Function ;
                    fno:example ( [ a fno:Test ;
                        dcterms:description ?description ;
                        rdfs:seeAlso ?seeAlso ;
                        fno:expression ?expression ;
                        fno:result ?result ] ) .
            }
            """
        query = string.Template(query).substitute(FUNCTION=str(func.s))
        for example in g.query(query):
            e = """
<div class=example>
    <p>$DESCRIPTION
    <p><b>Formula:</b>
    ```$EXPRESSION```
    <p><b>Result:</b>
    ```$RESULT```
</div>            
"""         
            url_param = urllib.parse.quote_plus(example.expression.strip(), safe='', encoding=None, errors=None)
            example_url = editor_uri + url_param

            expression = string.Template(e).substitute(DESCRIPTION=str(example.description), EXPRESSION=str(example.expression), RESULT=str(example.result))
            md_string += "[Try in Notation3 Editor &#128640;](" + example_url + ")\n\n"            
            md_string += expression + "\n"        
            
            

In [ ]:
template = open('index_TEMPLATE.bs').read()
with open('index.bs', 'w+') as f:
    string = template.replace('{{__CONTENT__}}', md_string)
    f.write(string)

In [12]:
print(md_string)

## crypto ## {#crypto}
### crypto:sha ### {#sha}
Gets as object the SHA-1 hash of the subject.

`true` if and only if `$o` is the SHA-1 hash of `$s`.

#### Schema(s) #### {#sha_schemas} 
<i>`( $s )+  crypto:sha  $o- `</i>

#### Examples #### {#sha_examples}
## math ## {#math}
### math:product ### {#product}
Calculates the object as the product of the numbers given in the subject list.

`true` if and only if `$o` is the arithmetic product of all numbers `$s.i`

#### Schema(s) #### {#product_schemas} 
<i>`( $s )+  math:product  $o- `</i>

#### Examples #### {#product_examples}
### math:notGreaterThan ### {#notGreaterThan}
Checks whether the subject is a number that is not greater than the object.
You can use this as an equivalent of a lessThanOrEqual operator.

`true` if and only if `$s` is a number that is not greater than `$o`.

#### Schema(s) #### {#notGreaterThan_schemas} 
<i>`( $s )+  math:notGreaterThan  $o- `</i>

#### Examples #### {#notGreaterThan_examples}
[Try in Notation3 Edi